In [1]:
from pyspark import SparkConf, SparkContext
conf=SparkConf().setMaster("local").setAppName("MinTemperatures")
sc=SparkContext(conf=conf)

In [2]:
directory="C:\\Users\\daesi\\Downloads\\빅데이터 소스코드\\소스코드\\study_spark\\data"
filename="1800.csv"

In [3]:
lines = sc.textFile(f"file:///{directory}\\{filename}")
lines.collect()[:5]

['ITE00100554,18000101,TMAX,-75,,,E,',
 'ITE00100554,18000101,TMIN,-148,,,E,',
 'GM000010962,18000101,PRCP,0,,,E,',
 'EZE00100082,18000101,TMAX,-86,,,E,',
 'EZE00100082,18000101,TMIN,-135,,,E,']

In [19]:
def parseLine(line):
    fields=line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3])*0.1*(9.0/5.0)+32.0
    
    return (stationID, entryType, temperature)

In [20]:
parsedLines = lines.map(parseLine)
parsedLines.collect()[:3]

[('ITE00100554', 'TMAX', 18.5),
 ('ITE00100554', 'TMIN', 5.359999999999999),
 ('GM000010962', 'PRCP', 32.0)]

In [22]:
minTemps=parsedLines.filter(lambda x : "TMIN" in x[1])
minTemps.collect()[:5]

[('ITE00100554', 'TMIN', 5.359999999999999),
 ('EZE00100082', 'TMIN', 7.699999999999999),
 ('ITE00100554', 'TMIN', 9.5),
 ('EZE00100082', 'TMIN', 8.599999999999998),
 ('ITE00100554', 'TMIN', 23.72)]

In [23]:
stationTemps = minTemps.map(lambda x : (x[0], x[2]))
stationTemps.collect()[:3]

[('ITE00100554', 5.359999999999999),
 ('EZE00100082', 7.699999999999999),
 ('ITE00100554', 9.5)]

In [24]:
stationTemps.reduceByKey(lambda x,y : min(x,y)).collect()[:5]

[('ITE00100554', 5.359999999999999), ('EZE00100082', 7.699999999999999)]

In [25]:
sc.stop()